In [1]:
import sys
from pymargo.core import Engine
from pysdskv.client import *
import matplotlib.pyplot as plt
import numpy as np

In [2]:
engine = Engine('na+sm') 
# engine = Engine('ofi+tcp')

#server_addr = input()
#db_name = input()
#provider_id = input()

server_addr = "na+sm://5383/0"
provider_id = 1
db_name = "foo"

In [3]:
client = SDSKVClient(engine)
addr = engine.lookup(server_addr)
provider_handle = client.create_provider_handle(addr, provider_id)
db = provider_handle.open(db_name)

In [4]:
num_papi_counters = int(db.get("num_papi_counters"))
print(num_papi_counters)
papi_counters = []
for i in range(num_papi_counters):
    key = "papi_counter_" + str(i)
    papi_counters.append(db.get(key))

print(papi_counters)

1
['PAPI_BR_UCN']


In [6]:
db.put("ADD_PAPI_COUNTER_1","PAPI_L1_ICM")


In [9]:
num_papi_counters = int(db.get("num_papi_counters"))
print(num_papi_counters)
papi_counters = []
for i in range(2):
    key = "papi_counter_" + str(i)
    papi_counters.append(db.get(key))

print(papi_counters)

3


KeyError: 'papi_counter_1'

In [5]:
db.put("ADD_PAPI_COUNTER_1","PAPI_L1_LCM")

In [ ]:
num_ranks = int(db.get("numRanks"))
db.get("numRanks")

In [ ]:
num_papi_counters = int(db.get("num_papi_counters"))
papi_counters = []
for i in range(num_papi_counters):
    key = "papi_counter_" + str(i)
    papi_counters.append(db.get(key))

print(papi_counters)

In [ ]:
db.put("add_papi_counter_0","PAPI_L1_LDM")
db.put("add_papi_counter_1","PAPI_L1_ICM")

In [ ]:
num_papi_counters = int(db.get("num_papi_counters"))
print(num_papi_counters)
papi_counters = []
for i in range(num_papi_counters):
    key = "papi_counter_" + str(i)
    papi_counters.append(db.get(key))

print(papi_counters)

In [ ]:
num_timesteps = int(db.get("current_timestep"))-1
print(num_timesteps)

counters = []
count = 0
for p in papi_counters:
    r_counter = []
    for r in range(num_ranks):
        t_counter = []
        for t in range(num_timesteps):
            key = p + "_" + str(t) + "_" + str(r)
            value = db.get(key)
            t_counter.append(int(value))
        r_counter.append(t_counter)
    counters.append(r_counter)
    count =  count + 1
    
#print(counters)

timesteps = np.arange(num_timesteps)

plt.plot(timesteps, counters[0][0], label=papi_counters[0], linestyle='-', marker='o')
plt.plot(timesteps, counters[0][1], label=papi_counters[1], linestyle='-', marker='x')
plt.plot(timesteps, counters[0][2], label=papi_counters[2], linestyle='-', marker='.')
plt.legend()
plt.title("Papi counters")
print(counters[0])

In [ ]:
ranks = list(range(num_ranks))
timesteps = list(range(num_timesteps))

heatmap = counters[0]
print(heatmap)

fig, ax = plt.subplots()
im = ax.imshow(heatmap)

# We want to show all ticks...
ax.set_xticks(np.arange(len(timesteps)))
ax.set_yticks(np.arange(len(ranks)))

ax.set_xticklabels(timesteps)
ax.set_yticklabels(ranks)

ax.tick_params(top=True, bottom=False, labeltop=True, labelbottom=False)

    # Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=-30, ha="right", rotation_mode="anchor")


for edge, spine in ax.spines.items():
    spine.set_visible(False)
    
ax.set_xticks(np.arange(len(timesteps)+1)+.5, minor=True)
ax.set_yticks(np.arange(len(ranks)+1)-.5, minor=True)

ax.grid(which="minor", color="w", linestyle='-', linewidth=3)
ax.tick_params(which="minor", bottom=False, left=False)



# Loop over data dimensions and create text annotations.
for i in range(len(ranks)):
    for j in range(len(timesteps)):
        text = ax.text(j, i, heatmap[i][j], ha="center", va="center", color="w")

ax.set_title("PAPI_L1_DCM")

plt.ylabel('ranks')
plt.xlabel('timesteps')
plt.show()

In [ ]:
num_timesteps = int(db.get("current_timestep"))-1
counters = []
for t in range(num_timesteps):
    r_counter = []
    for r in range(num_ranks):
        key = "comp-time_ts_" + str(t) + "_" + str(r)
        value = db.get(key)
        r_counter.append(float(value))
    counters.append(r_counter)
print(counters)


each_rank = np.arange(num_ranks)


for t in range(num_timesteps):
    plt.bar(each_rank, counters[t])
    
    plt.title("timstep " + str(t))
    plt.tight_layout()
    plt.show()
    


In [ ]:
import itk
import vtk
from itkwidgets import view

In [ ]:
filename = "/home/pascal/projects/SciDAC/InSituWriters/build/testStructuredMPI_8.pvts"
reader = vtk.vtkXMLPStructuredGridReader()
reader.SetFileName(filename)
reader.Update()
grid = reader.GetOutput()
print(grid)

In [ ]:



scalar_range = grid.GetCellData().GetArray("_rank").GetRange()
print(scalar_range)
colorData = grid.GetCellData().GetArray("_rank")
print(colorData)
grid.GetCellData().SetScalars(colorData)
tableSize = grid.GetCellData().GetArray("_rank").GetNumberOfTuples()
lut1 = MakeLUTFromCTF(tableSize)

	# into graphics elements
mapper = vtk.vtkDataSetMapper()
mapper.SetInputConnection(reader.GetOutputPort())
mapper.SetScalarRange(scalar_range)
mapper.SetScalarModeToUseFieldData()

mapper.SetLookupTable(lut1)
mapper.Update()


# Create the Actor
actor = vtk.vtkActor()
actor.SetMapper(mapper)

In [ ]:
view(geometries=grid,)

In [ ]:
import pyvista as pv
print(filename)
mesh = pv.read(filename)